In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="cpu"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

#model.config.output_attentions = True  # 返回注意力权重
model.config.output_hidden_states = True  # 返回隐藏状态（可选）


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [3]:
output = model(input_ids=tokenizer("Hello, my name is", return_tensors="pt").input_ids,
      output_hidden_states=True,
      return_dict=True,
      use_cache=False
)

In [8]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [2]:
prompt = "列出2020年1月份全球财经大事件"
messages = [
    {"role": "system", "content": "你是一名财经内容助手"},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)



In [ ]:
generated_ids = model.generate(**model_inputs,max_new_tokens=512)

In [4]:

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

system
你是一名财经内容助手
user
列出2020年1月份全球财经大事件
assistant
以下是2020年1月份全球财经领域的重大事件：

1. 欧洲央行宣布降息至负利率，这是自金融危机以来的首次。
2. 美国联邦储备系统（Fed）宣布加息50个基点，将联邦基金目标利率提高到2.25%。这标志着美联储在历史上第三次上调基准利率。
3. 英镑兑美元汇率大幅下跌，创下近一个月来的新低。
4. 中国国家主席习近平出席二十国集团领导人峰会，并发表重要讲话，强调了全球经济和贸易问题的重要性。
5. 芝加哥联储宣布加息50个基点，以应对通胀压力。
6. 中国股市出现反弹，但市场情绪仍不稳。

这些事件反映了全球金融市场的变化，包括货币政策、经济数据和政治动态等。


In [28]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [34]:
len(generated_ids['sequences'][0])

248

In [47]:
generated_ids['past_key_values'][1][1].shape

torch.Size([1, 2, 247, 64])